In [1]:
import os 
import json
from django.contrib.gis.geos import GEOSGeometry
import time 
import shutil 

import numpy as np
from shapely.geometry import mapping
import pandas as pd
import fiona
import geopandas as gpd
from fiona.crs import to_string
from fiona.crs import from_epsg
from shapely.geometry import shape, Point, Polygon
from fiona.crs import to_string

from madmex.overlay.conversions import train_object_to_feature
from madmex.util.spatial import geometry_transform
from madmex.models import TrainClassification

Checar primero:

https://github.com/CONABIO/antares3-sandbox/blob/master/notebooks/agriculture_madmex_app/3a_assign_labels_and_insert_in_db_for_agriculture.ipynb

# 7) Random sample para cada categoría generada de Kmeans

**La muestra debe generarse de modo que incluya los polígonos dados por inecol más otros y que en total sean 1000**

Intersección y contención del geopandas dataframe con el shapefile:

In [2]:
input_file2='/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-13-2020/47_-35/02-13-2020_clusters_47_-35_nclusters_3_aoi.shp'

In [3]:
gdf_df_clusters = gpd.read_file(input_file2)

In [4]:
gdf_df_clusters.head()

,features_i,preds,geometry
0,345111,1,"POLYGON ((3330680.000 714220.000, 3330690.000 ..."
1,345109,1,"POLYGON ((3330690.000 714180.000, 3330700.000 ..."
2,345110,1,"POLYGON ((3330700.000 714180.000, 3330710.000 ..."
3,344752,0,"POLYGON ((3330640.000 714230.000, 3330670.000 ..."
4,344418,1,"POLYGON ((3330730.000 714180.000, 3330750.000 ..."


In [5]:
len(gdf_df_clusters.index)

73032

In [6]:
gdf_df_clusters.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True}

In [7]:
#crs = {'proj': 'lcc',
# 'lat_1': 17.5,
# 'lat_2': 29.5,
# 'lat_0': 12,
# 'lon_0': -102,
# 'x_0': 2500000,
# 'y_0': 0,
# 'datum': 'WGS84',
# 'units': 'm',
# 'no_defs': True,
# 'wktext': True}

In [8]:
crs = gdf_df_clusters.crs

In [9]:
nclusters=len(gdf_df_clusters['preds'].unique())

In [10]:
nclusters

3

**Leer puntos de cultivos ya registrados con su valor numérico en la tabla TrainClassification**

Obtener extent en coordenadas geográficas de polígonos clúster

https://gis.stackexchange.com/questions/266730/filter-by-bounding-box-in-geopandas

http://geopandas.org/set_operations.html

In [11]:
bbox = gdf_df_clusters.total_bounds

In [12]:
p1 = Point(bbox[0], bbox[3])
p2 = Point(bbox[2], bbox[3])
p3 = Point(bbox[2], bbox[1])
p4 = Point(bbox[0], bbox[1])
p1 = shape(geometry_transform(mapping(p1),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                              crs_in=crs))
p2 = shape(geometry_transform(mapping(p2),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                              crs_in=crs))
p3 = shape(geometry_transform(mapping(p3),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                              crs_in=crs))
p4 = shape(geometry_transform(mapping(p4),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                              crs_in=crs))
np1 = (p1.coords.xy[0][0], p1.coords.xy[1][0])
np2 = (p2.coords.xy[0][0], p2.coords.xy[1][0])
np3 = (p3.coords.xy[0][0], p3.coords.xy[1][0])
np4 = (p4.coords.xy[0][0], p4.coords.xy[1][0])

In [13]:
bb_polygon = Polygon([np1, np2, np3, np4])

In [14]:
bb_polygon_for_query = GEOSGeometry(json.dumps(mapping(bb_polygon)))

In [15]:
bb_polygon_for_query

In [16]:
training_set = 'cultivos2019_inecol'

In [17]:
query_set = TrainClassification.objects.filter(train_object__the_geom__intersects=bb_polygon_for_query,
                                               training_set=training_set).prefetch_related('train_object', 'interpret_tag')

In [18]:
len(query_set)

29

In [19]:
to_string(crs)

'+datum=WGS84 +lat_0=12 +lat_1=17.5 +lat_2=29.5 +lon_0=-102 +no_defs +proj=lcc +units=m +x_0=2500000 +y_0=0'

In [20]:
fc_cultivos = [train_object_to_feature(x, to_string(crs)) for x in query_set]

In [21]:
fc_cultivos[0]

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [3336291.607239087, 676191.0055632746]},
 'properties': {'class': 19}}

In [22]:
# Build the GeoDataFrame from Fiona Collection
gdf_poi = gpd.GeoDataFrame.from_features([feature for feature in fc_cultivos], crs=crs)


In [23]:
gdf_poi.head()

,geometry,class
0,POINT (3336291.607 676191.006),19
1,POINT (3328436.983 678681.598),19
2,POINT (3335007.105 684938.439),14
3,POINT (3328547.488 684911.981),19
4,POINT (3330987.932 683832.847),19


In [24]:
len(gdf_poi.index)

29

In [25]:
gdf_poi.head()

,geometry,class
0,POINT (3336291.607 676191.006),19
1,POINT (3328436.983 678681.598),19
2,POINT (3335007.105 684938.439),14
3,POINT (3328547.488 684911.981),19
4,POINT (3330987.932 683832.847),19


**Interesan los id's que intersectan los puntos pues se extraerán tales id's del gdf_df_clusters**

Obtener `features_id` de `gdf_df_clusters`  que intersectan `gdf_poi`

In [26]:
gdf_df_clusters_poi_intersects = gpd.sjoin(gdf_df_clusters.reset_index(drop=True),
                                           gdf_poi.reset_index(drop=True),
                                           op='intersects',
                                           how='inner')

In [27]:
gdf_df_clusters_poi_intersects.head()

,features_i,preds,geometry,index_right,class
7719,292313,1,"POLYGON ((3328920.000 708100.000, 3328930.000 ...",27,19
32638,161895,0,"POLYGON ((3334560.000 693500.000, 3334590.000 ...",23,14
33025,159855,0,"POLYGON ((3329390.000 693260.000, 3329400.000 ...",10,14
33511,156753,0,"POLYGON ((3334660.000 692880.000, 3334670.000 ...",28,19
33582,156139,2,"POLYGON ((3332030.000 692750.000, 3332040.000 ...",17,14


In [28]:
len(gdf_df_clusters_poi_intersects.index)

28

In [29]:
gdf_df_clusters_keep = gpd.GeoDataFrame()

In [30]:
gdf_df_clusters_keep['features_i'] = gdf_df_clusters_poi_intersects['features_i']
gdf_df_clusters_keep['preds'] = gdf_df_clusters_poi_intersects['preds']
gdf_df_clusters_keep['geometry'] = gdf_df_clusters_poi_intersects['geometry']
gdf_df_clusters_keep['class'] = gdf_df_clusters_poi_intersects['class']

In [31]:
gdf_df_clusters_keep.head()

,features_i,preds,geometry,class
7719,292313,1,"POLYGON ((3328920.000 708100.000, 3328930.000 ...",19
32638,161895,0,"POLYGON ((3334560.000 693500.000, 3334590.000 ...",14
33025,159855,0,"POLYGON ((3329390.000 693260.000, 3329400.000 ...",14
33511,156753,0,"POLYGON ((3334660.000 692880.000, 3334670.000 ...",19
33582,156139,2,"POLYGON ((3332030.000 692750.000, 3332040.000 ...",14


In [32]:
len(gdf_df_clusters_keep.index)

28

In [33]:
gdf_df_clusters_keep.crs = crs

In [34]:
gdf_df_clusters_keep.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True}

**If number of rows of `gdf_df_clusters_keep` are less than 1000 then filter_out rows that don't intersect points and then upsample**

In [35]:
t_keep_rows = tuple(gdf_df_clusters_keep['features_i'])

In [36]:
len(t_keep_rows)

28

In [37]:
t_keep_rows[0:5]

(292313, 161895, 159855, 156753, 156139)

In [38]:
gdf_df_clusters_to_sample = gdf_df_clusters.loc[~gdf_df_clusters['features_i'].isin(t_keep_rows)]

In [39]:
gdf_df_clusters_to_sample.head()

,features_i,preds,geometry
0,345111,1,"POLYGON ((3330680.000 714220.000, 3330690.000 ..."
1,345109,1,"POLYGON ((3330690.000 714180.000, 3330700.000 ..."
2,345110,1,"POLYGON ((3330700.000 714180.000, 3330710.000 ..."
3,344752,0,"POLYGON ((3330640.000 714230.000, 3330670.000 ..."
4,344418,1,"POLYGON ((3330730.000 714180.000, 3330750.000 ..."


In [40]:
len(gdf_df_clusters_to_sample.index)

73004

In [41]:
len(gdf_df_clusters_keep.index)+len(gdf_df_clusters_to_sample.index)

73032

In [42]:
len(gdf_df_clusters.index)

73032

**Extraer muestra**

Using https://stackoverflow.com/questions/41345289/getting-a-random-sample-in-python-dataframe-by-category

In [43]:
gdf_df_clusters_to_sample.head()

,features_i,preds,geometry
0,345111,1,"POLYGON ((3330680.000 714220.000, 3330690.000 ..."
1,345109,1,"POLYGON ((3330690.000 714180.000, 3330700.000 ..."
2,345110,1,"POLYGON ((3330700.000 714180.000, 3330710.000 ..."
3,344752,0,"POLYGON ((3330640.000 714230.000, 3330670.000 ..."
4,344418,1,"POLYGON ((3330730.000 714180.000, 3330750.000 ..."


In [44]:
nmultipolygons=len(gdf_df_clusters_to_sample.index)

In [45]:
nmultipolygons

73004

In [46]:
total_mpolygons = 1000
if total_mpolygons>len(t_keep_rows):
    nsample=total_mpolygons-len(t_keep_rows)
else:
    nsample=total_mpolygons

In [47]:
nsample

972

In [48]:
nclusters

3

In [49]:
if nmultipolygons>nsample:
    nsample_per_cluster=int(nsample/nclusters)+1
    nsample=nsample_per_cluster*nclusters
else:
    nsample_per_cluster=int(nmultipolygons/nclusters)+1
    nsample=nsample_per_cluster*nclusters
 

In [50]:
(nmultipolygons,nsample_per_cluster,nsample_per_cluster*nclusters,nsample)

(73004, 325, 975, 975)

In [51]:
gdf_df_clusters_sample=gdf_df_clusters_to_sample.groupby('preds').apply(lambda s: s.sample(min(len(s),
                                                                                               nsample_per_cluster),
                                                                                          random_state=1))

In [52]:
gdf_df_clusters_sample.head()

features_i  preds  \
preds                            
0     58242       59227      0   
      17140      234577      0   
      71930        3290      0   
      48895       92270      0   
      15025      244855      0   

                                                      geometry  
preds                                                           
0     58242  POLYGON ((3333670.000 682490.000, 3333700.000 ...  
      17140  POLYGON ((3333390.000 701740.000, 3333400.000 ...  
      71930  POLYGON ((3339660.000 676470.000, 3339670.000 ...  
      48895  POLYGON ((3334430.000 686030.000, 3334470.000 ...  
      15025  POLYGON ((3331790.000 702830.000, 3331800.000 ...

In [53]:
len(gdf_df_clusters_sample.index)

975

In [54]:
gdf_df_clusters_sample['preds'].value_counts()

2    325
1    325
0    325
Name: preds, dtype: int64

In [55]:
gdf_df_clusters_sample.reset_index(drop=True,inplace=True)

In [56]:
gdf_df_clusters_sample.head()

,features_i,preds,geometry
0,59227,0,"POLYGON ((3333670.000 682490.000, 3333700.000 ..."
1,234577,0,"POLYGON ((3333390.000 701740.000, 3333400.000 ..."
2,3290,0,"POLYGON ((3339660.000 676470.000, 3339670.000 ..."
3,92270,0,"POLYGON ((3334430.000 686030.000, 3334470.000 ..."
4,244855,0,"POLYGON ((3331790.000 702830.000, 3331800.000 ..."


In [57]:
len(gdf_df_clusters_sample.index)

975

**Union of sample and keep polygons**

In [58]:
gdf_sampled_keep = pd.concat([gdf_df_clusters_sample,gdf_df_clusters_keep],
                             ignore_index=True,sort=False).pipe(gpd.GeoDataFrame)

In [59]:
gdf_sampled_keep.head()

,features_i,preds,geometry,class
0,59227,0,"POLYGON ((3333670.000 682490.000, 3333700.000 ...",NaN
1,234577,0,"POLYGON ((3333390.000 701740.000, 3333400.000 ...",NaN
2,3290,0,"POLYGON ((3339660.000 676470.000, 3339670.000 ...",NaN
3,92270,0,"POLYGON ((3334430.000 686030.000, 3334470.000 ...",NaN
4,244855,0,"POLYGON ((3331790.000 702830.000, 3331800.000 ...",NaN


In [60]:
len(gdf_sampled_keep.index)

1003

In [61]:
gdf_sampled_keep['class'].isna()

0        True
1        True
2        True
3        True
4        True
        ...  
998     False
999     False
1000    False
1001    False
1002    False
Name: class, Length: 1003, dtype: bool

In [62]:
if sum(gdf_sampled_keep['class'].isna())>0:
    gdf_sampled_keep.loc[gdf_sampled_keep['class'].isna(),'class'] = ''

In [63]:
sum(gdf_sampled_keep['class'].isna())

0

In [64]:
gdf_sampled_keep.head()

,features_i,preds,geometry,class
0,59227,0,"POLYGON ((3333670.000 682490.000, 3333700.000 ...",
1,234577,0,"POLYGON ((3333390.000 701740.000, 3333400.000 ...",
2,3290,0,"POLYGON ((3339660.000 676470.000, 3339670.000 ...",
3,92270,0,"POLYGON ((3334430.000 686030.000, 3334470.000 ...",
4,244855,0,"POLYGON ((3331790.000 702830.000, 3331800.000 ...",


In [65]:
len(gdf_sampled_keep.index)

1003

# 8) Write results to file

In [66]:
date_today = '02-13-2020'
dc_tile = '47_-35'
features_string = 'ndvi-gndvi-avi'
path_result_dc_tile = '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/' + features_string + '/' + date_today + '/' + dc_tile + '/'
total_mpolygons = len(gdf_sampled_keep.index)

In [67]:
total_mpolygons

1003

In [68]:
path_result_dc_tile

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-13-2020/47_-35/'

**Geopackage**

In [69]:
path_results_geopackage = '/home/madmex_user/agriculture_geopackages/features_computed/' + features_string + '/'

In [70]:
path_results_geopackage

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi-gndvi-avi/'

In [71]:
path_result_geopackage = os.path.join(path_results_geopackage, date_today 
                                       + '/' + dc_tile + '/')

In [72]:
path_result_geopackage

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi-gndvi-avi/02-13-2020/47_-35/'

In [73]:
if not os.path.exists(path_result_geopackage):
    os.makedirs(path_result_geopackage)

In [74]:
clusters_filename = path_result_geopackage + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_sample_%d' % total_mpolygons + '_with_class_aoi.gpkg'

In [75]:
clusters_filename

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi-gndvi-avi/02-13-2020/47_-35/02-13-2020_clusters_47_-35_nclusters_3_sample_1003_with_class_aoi.gpkg'

In [76]:
layer_name = date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_sample_%d' % total_mpolygons + '_with_class_aoi'

In [77]:
layer_name

'02-13-2020_clusters_47_-35_nclusters_3_sample_1003_with_class_aoi'

In [78]:
gdf_sampled_keep.crs

In [79]:
gdf_sampled_keep.crs = crs

In [80]:
gdf_sampled_keep.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True}

In [81]:
start_time=time.time()

gdf_sampled_keep.to_file(clusters_filename,
                         layer=layer_name,
                         driver='GPKG')
end_time=time.time()

In [82]:
secs = end_time-start_time
secs

0.4114210605621338

In [83]:
shutil.move(clusters_filename,path_result_dc_tile)

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-13-2020/47_-35/02-13-2020_clusters_47_-35_nclusters_3_sample_1003_with_class_aoi.gpkg'

**Shapefile**

In [85]:
clusters_filename = path_result_dc_tile + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_sample_%d' % total_mpolygons + '_with_class_aoi.shp'

In [86]:
clusters_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-13-2020/47_-35/02-13-2020_clusters_47_-35_nclusters_3_sample_1003_with_class_aoi.shp'

In [87]:
gdf_sampled_keep.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [88]:
gdf_sampled_keep.crs = crs

In [89]:
gdf_sampled_keep.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [90]:
start_time=time.time()
gdf_sampled_keep.to_file(clusters_filename)
end_time=time.time()

In [91]:
secs = end_time-start_time
secs

0.3763890266418457

# 12) Registrar segmentos con etiquetas automáticas en trainclassificationlabeledbyapp con esquema "automatic_3_classes"

**Command if shapefile of reference do have labels:**

In [92]:
%%bash
/home/madmex_user/.local/bin/antares ingest_training_from_vector \
/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-13-2020/47_-35/02-13-2020_clusters_47_-35_nclusters_3_sample_1003_with_class_aoi.shp \
--scheme automatic_3_classes --year 2020 --name agricultura_7_classes_2359_with_class_added \
--field preds --app --train_interpreted --scheme_interpreted agricultura_cultivos2019 --field_interpreted class --dc_tile 47_-35

Command line arguments: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'input_file': '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-13-2020/47_-35/02-13-2020_clusters_47_-35_nclusters_3_sample_1003_with_class_aoi.shp', 'scheme': 'automatic_3_classes', 'field': 'preds', 'name': 'agricultura_7_classes_2359_with_class_added', 'year': '2020', 'app': True, 'train_interpreted': True, 'scheme_interpreted': 'agricultura_cultivos2019', 'field_interpreted': 'class', 'dc_tile': '47_-35'}
Command execution is done in 37.8099160194397 seconds.
2020-02-14 15:32:21,094 - madmex.management.base - base execute: Command execution is done in 37.8099160194397 seconds.


**Command if shapefile of reference don't have labels:**

In [81]:
#%%bash
#/home/madmex_user/.local/bin/antares ingest_training_from_vector \
#/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000_with_class.shp \
#--scheme automatic_4_classes --year 2020 --name agricultura_4_clases_sample_1000_with_class --field preds --app

Command line arguments: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'input_file': '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000_with_class.shp', 'scheme': 'automatic_4_classes', 'field': 'preds', 'name': 'agricultura_4_clases_sample_1000_with_class', 'year': '2020', 'app': True}
Command execution is done in 13.483619928359985 seconds.
2020-02-06 20:55:45,824 - madmex.management.base - base execute: Command execution is done in 13.483619928359985 seconds.
